# User ranking in base alle skills

In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

cu = db.users.find()
cs = db.shots.find()
ct = db.teams.find()
cutfs = db.user_team_first_shot.find()
cr = db.shots_ranking.find()

In [2]:
usernames_no_repeated = []
for user in cutfs:
    if user['user'] not in usernames_no_repeated:
        usernames_no_repeated.append(user['user'])

In [10]:
skills_found = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/skills_frequences.txt", "r") as ins:
    for line in ins:
        skill = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'skill': skill, 'count': int(count)}
        skills_found.append(elem)

In [14]:
def n_most_frequent(n, skill_list):
    
    skills = []
    
    max_skill_prev = {'count':1000000}
    for i in range(0,n):
        max = 0
        max_skill = {}
        for t in skill_list:
            if int(t['count']) > max and int(t['count']) < int(max_skill_prev['count']):
                max = int(t['count'])
                max_skill = t
                
        
        max_skill_prev = max_skill
        skills.append(max_skill)
    return skills

In [16]:
skills_str = []
for skill in n_most_frequent(10, skills_found):
    skills_str.append(skill['skill'])
skills_str

['web design',
 'ui',
 'ux',
 'illustration',
 'branding',
 'product design',
 'ui design',
 'graphic design',
 'ux design',
 'design']

In [ ]:
%%time

ranking = []

i=0

for u in usernames_no_repeated:
    active_user = db.skills.find({'user': u})
    if active_user.count() > 0: 
        user_rank = 0
        for s in active_user[0]['skills']:
            if s in skills_str:
                user_rank = user_rank + 1

        elem = {'user': active_user[0]['user'], 'rank': user_rank}
        ranking.append(elem)
    
    i=i+1
    print i

In [ ]:
i=0
for r in ranking:
    db.user_ranking.insert_one(r)
    i=i+1
    print i

### Funzione che restitusce il rank di uno user

In [32]:
def user_rank(user):
    return db.user_ranking.find({'user': user})[0]['rank']

In [33]:
user_rank('hellohelloteam')

6